## Building AI assistants that scale using machine learning and open source tools

![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaX3LNhGcAe1HnPZSuWS0oH6af0LJHXcH7If1sQgLCFAT1chNGFg)

In this workshop, you will learn how to build your own conversational AI assistant using machine learning and real conversational data. The goal of this workshop is to walk you through the process of building an ML-powered assistant from scratch and build an actual assistant which you can improve later.


There are no additional requirements to run this notebook, but if you encounter any issues or have more questions about the content included here, feel free send a message to an author of this exercise at juste@gmail.com

### Introduction

### Introdução

During the course of this 3-hour workshop, you will go through each stage of the chatbot development and build an assistant capable of recommending talks to attend in this conference and answer the questions about the event. Below is an example conversation your assistant will be able to handle:

U: Hello  
A: Hello! I am a conference assistant ready to help you find talks to attend or anwers any questions you have.  
U: What session would you recommend for Data Scientists today?  
A: You can attend 'Building AI assistnats that scale using machine learning and open source tools' session!  
U: Who is the speaker and how long is the session?  
A: The speaker is Justina. The session is 3 hours long.  
U: Thanks!  
A: You are very welcome!  



      

The workshop consists of the following stages:

**0. Intro:**  
    0.1 Setup and installation 
      
**1. Stage 1: Natural language understanding:**  
    1.1. Designing the happy path  
    1.2. Generating the NLU training examples  
    1.3. Designing the training pipeline  
    1.4. Training the first NLU model  
    1.5.  Handling out-of-scope inputs  
    1.6. Adding sinonyms  
    1.7. Adding multi-intents  
    1.8. Re-training and thesting the updated NLU model  
      
**2. Stage 2: Dialogue management model:**  
    2.1. Designing training stories  
    2.2. Setting up the backend component  
    2.3. Creating a custom action  
    2.4. Defining the domain  
    2.5. Training the dialogue model  
    2.6. Testing the dialogue model   
    2.7. Handling out-of-scope conversations  
    2.8. Adding stories with multi-intents  
    2.9. Evaluating dialogue model  
      
**3. Stage 3: Closing the feedback loop:**  
    3.1. Improving the assistant using the interactive learning   
    3.2. Storing conversation history    
    3.3. Connecting the assistant to the outside world  
   

## 0. Intro
In this section, we will install all the necessary dependencies needed to successfully run this exercise.
### 0.1. Setup and installation
The best way to insall the necessary modules is to use the requirements.txt file. After creating a virtual environment, run:

**pip install -r requirements.txt**

Throughout this workshop, we will use only open source tools. The code block below checks if Rasa NLU and Rasa Core have been installed suffessfully.

## 0. Introdução
Nesta seção, instalaremos todas as dependências necessárias para executar este exercício com sucesso.
### 0.1. Instalação e instalação
A melhor maneira de instalar os módulos necessários é usar o arquivo requirements.txt. Depois de criar um ambiente virtual, execute:

**pip install -r requirements.txt**

Ao longo deste workshop, usaremos apenas ferramentas de código aberto. O bloco de código abaixo verifica se o Rasa NLU e o Rasa Core foram instalados sem problemas.

In [ ]:
import rasa_nlu
import rasa_core
import warnings
warnings.filterwarnings('ignore')


print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))

### 0.2. Translate tutorial

The good way to read a tutorial is in your native language, jupyter available for us a extension with Google translate.

First verify the extension, just run in your terminal ```jupyter nbextension list```, and check the extension nbTranslate
if you have not the extension installed in your computer, just **click on cell below** and **press ctrl + enter**.

In [ ]:
import os


os.system("jupyter nbextension install https://rawgit.com/jfbercher/jupyter_nbTranslate/master/nbTranslate.zip --user")
os.system("jupyter nbextension enable nbTranslate/main")

if you already have the extension in your computer:

In [2]:
import os


os.system("jupyter nbextension enable nbTranslate/main")

0

After this command, refresh your page to appear the settings translate on tool bar.

## 1. Natural Language Understanding 

In this section, you will enable your assistant to understand the user inputs by building a Rasa NLU model. This model will take unstructured user inputs and extract structured data in a form of intents and entities:  
- *intent* - a label which represents the overall intention of the user 's input
- *entity* - important detail which an assistant should extract and use to steer the conversation

### 1.1. Designing a happy path

A good starting point is to define a happy path first. A happy path is a conversation flow where the user provides all the required information and allows the assistant to lead the conversation.


### 1.2. Designing the NLU training data

To train the NLU model you will need some labeled training data. Rasa NLU training data samples consist of the following components:  
- intent label which starts with a prefix *
- examples of text inputs which correspond to that label
- entities which follow the format *[entity_value] (entity_label)*

We will start by generating some training data examples by hand. For a completed data file check out the *helper_files/nlu_data.md* in the repository of this exercise.

In [ ]:
nlu_md = """

## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:recommend_session
- What presentation would you recommend to [data scientists](relevant_audience)?
- Which talks are relevant to people in [Machine Learning](relevant_audience) field?
- I work as a [product manager](relevant_audience). What sessions would you recommend for me to attend today?
- Are the any talks you could recommend to [machine learning](relevant_audience) folks to attend tomorrow?
- Which talks today are relevant to [developers](relevant_audience)?

## intent:speaker
- Who is the speaker?
- And who's presenting?
- What's the name of the presenter?
- Who's presenting?
- Who's speaking?
- The name of the speaker?

## intent:length
- How long is the session?
- And what's the length of this?
- How long is this session?
- Can you tell me how long this session is?
- Is the session long?

## intent:abstract
- Show me the abstract
- Can you give me more details about this talk?
- Is there a description of this presetnation?
- Can you show me an abstract of this talk?
- Show me the abstract, please
- Can you show me the summary of the talk?
- What this talk will be about?

## intent:thanks
- Thank you.
- very useful. thank you so much!
- Thanks
- thanks a lot
- thank you so much

## intent:inform
- to [Data Scientists](relevant_audience)
- relevant to [machine learning engineers](relevant_audience)
- for [product](relevant_audience) people
"""

%store nlu_md > nlu.md



## 1.3 Designing the training pipeline

Once the training data is ready, we can define the NLU model. We can do that by constructing the processing pipeline which defines how structured data will be extracted from unstructured user inputs: how the sentences will be tokenized, what intent classifier will be used, what entity extraction model will be used, etc. Each component in a training pipeline is trained one after another and can take inputs from the previously defined component as well as pass some information to subsequent ones.

In [ ]:
configuration = """
language: "en"

pipeline:
- name: "tokenizer_whitespace"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_count_vectors"     # transform the sentence into a vector representation
- name: "intent_classifier_tensorflow_embedding"   # intent classifier
""" 

%store configuration > config.yml

## 1.4 Training the first Rasa NLU Model
Now, we're going to train the NLU model to recognise user inputs, so that when you send a message like "hello" to your bot, it will recognise this as a "greet" intent. Let's define the training function:

In [ ]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

def train_nlu_model():
    # loading the nlu training samples
    training_data = load_data("nlu.md")

    # trainer to educate our pipeline
    trainer = Trainer(config.load("config.yml"))

    # train the model!
    interpreter = trainer.train(training_data)

    # store it for future use
    model_directory = trainer.persist("./models/current", fixed_model_name="nlu")
    
    return interpreter, model_directory



Finally, let's train the model using the previously defined data and model configuration:

In [ ]:
interpreter, model_directory = train_nlu_model()

## Testing the model

We have trained the first version of our NLU model! Let's test it on various inputs:

In [ ]:
import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper function to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

#change the input message with your prefered inputs
input_message = "What talks would you recommend to data scientists?"
pprint(interpreter.parse(input_message))

## Handling out-of-scope inputs
When dealing with conversational AI, out-of-scope user inputs are very common challenge. These inputs represent the user requests which have nothing to do with the assistant's job. While it's very challenging to provide a sensible answer to each out-of-scope input, it's important to enable your assistant to identify such inputs and guide the user back to the conversation. First, let's enable our assistant to identify out-of-scope inputs. To do that, we will add a new intent called *out-of-scope* to our training dataset and provde some corresponding inputs:

In [ ]:
nlu_md = """


## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:recommend_session
- What presentation would you recommend to [data scientists](relevant_audience)?
- Which talks are relevant to people in [Machine Learning](relevant_audience) field?
- I work as a [product manager](relevant_audience). What sessions would you recommend for me to attend today?
- Are the any talks you could recommend to [machine learning](relevant_audience) folks to attend tomorrow?
- Which talks today are relevant to [developers](relevant_audience)?

## intent:speaker
- Who is the speaker?
- And who's presenting?
- What's the name of the presenter?
- Who's presenting?
- Who's speaking?
- The name of the speaker?

## intent:length
- How long is the session?
- And what's the length of this?
- How long is this session?
- Can you tell me how long this session is?
- Is the session long?

## intent:abstract
- Show me the abstract
- Can you give me more details about this talk?
- Is there a description of this presetnation?
- Can you show me an abstract of this talk?
- Show me the abstract, please
- Can you show me the summary of the talk?
- What this talk will be about?

## intent:thanks
- Thank you.
- very useful. thank you so much!
- Thanks
- thanks a lot
- thank you so much

## intent:inform
- to [Data Scientists](relevant_audience)
- relevant to [machine learning engineers](relevant_audience)
- for [product](relevant_audience) people

## intent:out-of-scope
- I want pizza
- please help with my ice cream it's dripping
- no wait go back i want a dripping ice cream but a cone that catches it so you can drink the ice cream later
- i want a non dripping ice cream
- hey little mama let em whisper in your ear
- someone call the police i think the bot died
- show me a picture of a chicken
- neither
- I want french cuisine
- i am hungry
- restaurants
- restaurant
- you're a loser lmao
- can i be shown a gluten free restaurant
- i don't care!!!!
- i do not care how are you
- again?
- oh wait i gave you my work email address can i change it?
- hang on let me find it
- stop it, i do not care!!!
- really? you're so touchy?
- how come?
- I changed my mind
- what?
- did i break you

"""

%store nlu_md > nlu.md


Let's retrain the model and see how it deals with out-of-scope inputs now:

Vamos treinar novamente o modelo e ver como ele lida com as entradas fora do escopo agora:

In [ ]:
interpreter, model_directory = train_nlu_model()

In [ ]:
input_message = "I want pizza"
pprint(interpreter.parse(input_message))

## 1.6 Adding synonyms

Synonyms are a very useful Rasa NLU feature which maps extracted entities to the same name. It's used when some extracted values have to be normalised so that they could be used to query the database or make an API call. In our example, the occupation of the relevant audience is a good candidate for the synonym because users can provide the same occupation in a variety of different ways (for example, Machine Learning and ML). Let's update our training examples with synonyms.

In [ ]:
nlu_md = """

## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:recommend_session
- What presentation would you recommend to [data scientists](relevant_audience)?
- Which talks are relevant to people in [Machine Learning](relevant_audience:ML) field?
- I work as a [product manager](relevant_audience). What sessions would you recommend for me to attend today?
- Are the any talks you could recommend to [machine learning](relevant_audience:ML) folks to attend tomorrow?
- Which talks today are relevant to [developers](relevant_audience)?

## intent:speaker
- Who is the speaker?
- And who's presenting?
- What's the name of the presenter?
- Who's presenting?
- Who's speaking?
- The name of the speaker?

## intent:length
- How long is the session?
- And what's the length of this?
- How long is this session?
- Can you tell me how long this session is?
- Is the session long?

## intent:abstract
- Show me the abstract
- Can you give me more details about this talk?
- Is there a description of this presetnation?
- Can you show me an abstract of this talk?
- Show me the abstract, please
- Can you show me the summary of the talk?
- What this talk will be about?

## intent:thanks
- Thank you.
- very useful. thank you so much!
- Thanks
- thanks a lot
- thank you so much

## intent:inform
- to [Data Scientists](relevant_audience)
- relevant to [machine learning engineers](relevant_audience:ML)
- for [product](relevant_audience) people


## intent:out-of-scope
- I want pizza
- please help with my ice cream it's dripping
- no wait go back i want a dripping ice cream but a cone that catches it so you can drink the ice cream later
- i want a non dripping ice cream
- hey little mama let em whisper in your ear
- someone call the police i think the bot died
- show me a picture of a chicken
- neither
- I want french cuisine
- i am hungry
- restaurants
- restaurant
- you're a loser lmao
- can i be shown a gluten free restaurant
- i don't care!!!!
- i do not care how are you
- again?
- oh wait i gave you my work email address can i change it?
- hang on let me find it
- stop it, i do not care!!!
- really? you're so touchy?
- how come?
- I changed my mind
- what?
- did i break you
"""

%store nlu_md > nlu.md

To train the NLU model with synonyms, we have to add the synonyms component to the model pipeline:

In [ ]:
configuration = """
language: "en"

pipeline:
- name: "tokenizer_whitespace"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_count_vectors"     # transform the sentence into a vector representation
- name: "intent_classifier_tensorflow_embedding"   # intent classifier
- name: "ner_synonyms"     #trains synonyms component
""" 

%store configuration > config.yml

Now, let's retrain the NLU model and test the performace.

In [ ]:
interpreter, model_directory = train_nlu_model()

See how 'machine learning engineers' now gets mapped to 'ML':

In [ ]:
input_message = "For machine learning engineers"
pprint(interpreter.parse(input_message))

## 1.7 Implementing multi-intents

The NLU model we have built so far works pretty well, but it only supports inputs with only one intent per user input. In this step, we will use a tensorflow embedding model to enable the assistant to recognise multi-intents - more than one intention per user input. Let's start by defining multi-intents in our training data. Multi-intents are defined in a very similar way as regular intents, the only difference is that the label names consists of intent tokens and a character of your choice that separates them, for example **intent_token1+intent_token2**.

In [ ]:
nlu_md = """

## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:recommend_session
- What presentation would you recommend to [data scientists](relevant_audience)?
- Which talks are relevant to people in [Machine Learning](relevant_audience:ML) field?
- I work as a [product manager](relevant_audience). What sessions would you recommend for me to attend today?
- Are the any talks you could recommend to [machine learning](relevant_audience:ML) folks to attend tomorrow?
- Which talks today are relevant to [developers](relevant_audience)?

## intent:speaker
- Who is the speaker?
- And who's presenting?
- What's the name of the presenter?
- Who's presenting?
- Who's speaking?
- The name of the speaker?

## intent:length
- How long is the session?
- And what's the length of this?
- How long is this session?
- Can you tell me how long this session is?
- Is the session long?

## intent:abstract
- Show me the abstract
- Can you give me more details about this talk?
- Is there a description of this presetnation?
- Can you show me an abstract of this talk?
- Show me the abstract, please
- Can you show me the summary of the talk?
- What this talk will be about?

## intent:thanks
- Thank you.
- very useful. thank you so much!
- Thanks
- thanks a lot
- thank you so much

## intent:inform
- to [Data Scientists](relevant_audience)
- relevant to [machine learning engineers](relevant_audience:ML)
- for [product](relevant_audience) people


## intent:out-of-scope
- I want pizza
- please help with my ice cream it's dripping
- no wait go back i want a dripping ice cream but a cone that catches it so you can drink the ice cream later
- i want a non dripping ice cream
- hey little mama let em whisper in your ear
- someone call the police i think the bot died
- show me a picture of a chicken
- neither
- I want french cuisine
- i am hungry
- restaurants
- restaurant
- you're a loser lmao
- can i be shown a gluten free restaurant
- i don't care!!!!
- i do not care how are you
- again?
- oh wait i gave you my work email address can i change it?
- hang on let me find it
- stop it, i do not care!!!
- really? you're so touchy?
- how come?
- I changed my mind
- what?
- did i break you


## intent:speaker+length
 - Who is the presenter? Also, how long is the talk?
 - Who is the speaker and how long is the session?
 - Is the session long and who is presenting?
 - Do you know who is the presenter of the session? And how long is the session?
 - Is the talk long and who is presenting?
 - Who is the speaker? And how long is the talk?
"""

%store nlu_md > nlu.md


Next, let's modify the configuration of the model pipeline to use the tensorflow_embedding model with multi-intent support.

In [ ]:
configuration = """
language: "en"

pipeline:
- name: "tokenizer_whitespace"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_count_vectors"     # transform the sentence into a vector representation
- name: "intent_classifier_tensorflow_embedding"   # intent classifier
  intent_tokenization_flag: true # sets multi-intent tokenization
  intent_split_symbol: "+"       # sets which symbol should be used for tokenization
- name: "ner_synonyms"   # trains synonyms component 
""" 

%store configuration > config.yml

Let's retrain the model with the new pipeline and test the performance:

In [ ]:
interpreter, model_directory = train_nlu_model()

See how a two-question input now gets recognised as a multi-intent:

In [ ]:
input_message = "Who is the speakers and how long is the session?"
pprint(interpreter.parse(input_message))

## 1.8 Evaluating the NLU model


Testing the model on various inputs is a good way to get high-level insights into the performance of the model. However, it's a time consuming and quite a tedious way of testing. Instead of evaluating the model by hand, it can also be evaluated on a test data set (though for simplicity we are going to use the same for test and train):

In [ ]:

from rasa_nlu.evaluate import run_evaluation
import IPython
from IPython import display

run_evaluation("nlu.md", model_directory)

Congratulations! You have just implemented the natural language understanding part of your assistant which means that your assistant can now understand you. In the second part of this workshop, we will delve into the next stage of the chatbot development - dialogue management.

# 2. Dialogue Management


In this section of this workshop you will build a machine learning-based dialogue model which will enable your assistant to decide on how to respond to user inputs based on the state of the conversation. 

## 2.1 Designing the training stories

Let's start with generating the training data. Rasa Core models learn by observing real conversational data between the user and the assistant. The only important thing is that this data has to be converted into the Rasa Core format: user inputs have to be expressed as corresponding intents (and entities where necessary) while the responses of the assistant are expressed as action names. Each training story follows the format:  
- the story starts with a story name which has a prefix ##  
- intents, corresponding to user inputs, start with *  
- if NLU model extracts entities which should influence the predictions of the dialogue model, they have to be included in the stories using the following format: * intent{'entity_name':"entity_value"}  
- the responses of the bot start with -  
- the story ends with an empty line which marks the end of the story

In the next step of this tutorial, we will generate some training stories to cover the happy path. To see a complete training data example, check out the **data/stories.md** file of this repository.


In [ ]:
stories_md = """
## happy path all info 1               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* thanks
  - utter_thanks
  
  
## happy path all info 2               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks    
  

## happy path all info 3               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker
  - utter_speaker
* length
  - utter_length
* thanks
  - utter_thanks  


## happy path no relevant audience            
* greet              
  - utter_greet
* recommend_session
  - utter_ask_relevant_audience
* inform{"relevant_audience":"Data Scientists"}
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks 

"""

%store stories_md > stories.md

## 2.2 Setting up the backend component

We want to make our assistant engaging and fun. For that reason, we will enable it to answer the questions using the real data stored in a SQL database. For this exercise, the assistant will be able to pull information about the conference agenda, talks, and speakers. Let's take a look at how the data in a SQL database looks like.

In [ ]:
import sqlite3 as lite
import pandas as pd

conn = lite.connect('ConfDB.db')
data = pd.read_sql_query("SELECT * FROM agenda LIMIT 10", conn)
print(data)

## 2.3 Creating custom action

We are going to use the backend integration to enable our assistant to fetch the relevant data based on user's queries. For that, we will create custom actions which, when predicted, will collect necessary data and use it to steer the conversation further:

In [ ]:
actions = """
from rasa_core_sdk import Action
from rasa_core_sdk.events import SlotSet
import sqlite3 as lite
import pandas as pd
import time
from datetime import datetime
import random


class ActionRecommendTalk(Action):
    def name(self):
        return "action_recommend_session"
        
    def run(self, dispatcher, tracker, domain):
        conn = lite.connect('ConfDB.db')
        cur = conn.cursor()
        relevant_audience = tracker.get_slot('relevant_audience')
        cur.execute("SELECT * FROM agenda where relevant_audience like '%{}%'".format(relevant_audience))

        rows = cur.fetchall()
        ind = random.randint(0,len(rows))
        recommend_talk = list(rows[ind])
        title = recommend_talk[0]
        length = recommend_talk[8]
        speaker = recommend_talk[2]
        abstract = recommend_talk[4]
        
        dispatcher.utter_message("I would recommend you attend: {}".format(title))


        return [SlotSet("speaker", speaker), SlotSet("length",length), SlotSet("abstract", abstract)]

"""

%store actions > actions.py

## 2.4 Defining the domain

Once we have the training data in place, we can define the domain of our assistant. A domain defines the environment in which the assistant operates - what user inputs it should expect to see, what actions it should be able to predict, what information the assistant should store throughout the conversation.

In [ ]:
domain_yml = """
intents:
- greet
- goodbye
- recommend_session
- speaker
- length
- abstract
- speaker+length
- thanks
- out-of-scope
- inform



slots:
  relevant_audience:
    type: text
  venue:
    type: unfeaturized
  length:
    type: unfeaturized
  speaker:
    type: unfeaturized
  abstract:
    type: unfeaturized
    
entities:
- relevant_audience


actions:
- utter_greet
- utter_thanks
- utter_goodbye
- utter_length
- utter_abstract
- utter_speaker
- utter_ask_relevant_audience
- utter_out_of_scope
- utter_ask_other_questions
- action_recommend_session


templates:
  utter_greet:
    - text: "Hey! I am a conference assistant. I can help you find the sessions to attend, or answer conference-related questions."

  utter_thanks:
    - text: "You are very welcome!"

  utter_goodbye:
    - text: "See you later"
  
  utter_out_of_scope:
    - text: "Sorry, I can't help you with that"
  
  utter_ask_other_questions:
    - text: "Would you like to know anything else?"
    
  utter_ask_relevant_audience:
    - text: "What would be the relevant audience?"
  
  utter_speaker:
    - text: "The speaker is {speaker}."
  
  utter_length:
    - text: "The session is {length}"
    
  utter_abstract:
    - text: "{abstract}"

"""

%store domain_yml > domain.yml

## 2.5 Training the dialogue model

We now have all the components necessary to train the dialogue management model. The code cell below will train the model using the defined policy and store the model in a specified location for us to test later.

In [ ]:
from rasa_core.policies import KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

def train_dialogue():
    # loading our neatly defined training dialogues
    agent = Agent("domain.yml", policies=[MemoizationPolicy(), KerasPolicy(epochs=200, max_history = 6)])
    training_data = agent.load_data('stories.md')


    agent.train(
        training_data)

    agent.persist('models/dialogue')


In [ ]:
train_dialogue()

## 2.6 Testing the dialogue model

It's finally time for the most exciting part - testing the bot! Let's spin up the custom action server and we are ready to go. Open a new terminal and exacute the following command:

**python -m rasa_core_sdk.endpoint --actions actions**

In [ ]:
import IPython
from IPython.display import clear_output
from rasa_core.agent import Agent
from rasa_core.interpreter import NaturalLanguageInterpreter
from rasa_core.utils import EndpointConfig
import time

def load_assistant():
    messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
    interpreter = NaturalLanguageInterpreter.create(model_directory)
    endpoint = EndpointConfig('http://localhost:5055/webhook')
    agent = Agent.load('models/dialogue', interpreter=interpreter, action_endpoint = endpoint)

    print("Your bot is ready to talk! Type your messages here or send 'stop'")
    while True:
        a = input()
        if a == 'stop':
            break
        responses = agent.handle_text(a)
        for response in responses:
            print(response["text"])

In [ ]:
load_assistant()

## 2.7 Adding stories with multi-intents

Next, let's add a few stories with multi-intents. Such stories will follow a regular data format, the only thing is that we can include a couple of actions to be predicted by an assistant:

In [ ]:
stories_md = """
               
## happy path all info 1               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* thanks
  - utter_thanks
  
  
## happy path all info 2               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks    
  

## happy path all info 3               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker
  - utter_speaker
* length
  - utter_length
* thanks
  - utter_thanks  


## happy path no relevant audience            
* greet              
  - utter_greet
* recommend_session
  - utter_ask_relevant_audience
* inform{"relevant_audience":"Data Scientists"}
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks 
  
## multi-intents story1              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* thanks
  - utter_thanks
  
## multi-intents story2              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* abstract
  - utter_abstract
* thanks
  - utter_thanks
  


"""

%store stories_md > stories.md

In [ ]:
train_dialogue()

In [ ]:
load_assistant()

## 2.8 Adding out-of-scope inputs

Finally, let's design a story with out-of-scope user inputs. Here, it's important to enable an assistant to take charge of the conversation and guide the user back to the initial conversation. In our case, an assistant will let the user know that it cannot deal with the out-of-scope request and will offer other questions to be asked:

In [ ]:
stories_md = """
## happy path all info 1               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* thanks
  - utter_thanks
  
  
## happy path all info 2               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks    
  

## happy path all info 3               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker
  - utter_speaker
* length
  - utter_length
* thanks
  - utter_thanks  


## happy path no relevant audience            
* greet              
  - utter_greet
* recommend_session
  - utter_ask_relevant_audience
* inform{"relevant_audience":"Data Scientists"}
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks 

  
## multi-intents story1              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* thanks
  - utter_thanks
  
## multi-intents story2              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* abstract
  - utter_abstract
* thanks
  - utter_thanks
  
## multi-intents story2              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* abstract
  - utter_abstract
* thanks
  - utter_thanks
  
  
## multi-intents story2              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* abstract
  - utter_abstract
* thanks
  - utter_thanks
  
  
  
## out-of-scope input story 1            
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* out-of-scope
  - utter_out_of_scope
  - utter_ask_other_questions
* abstract
  - utter_abstract
* thanks
  - utter_thanks
  
## out-of-scope input story 2            
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* out-of-scope
  - utter_out_of_scope
  - utter_ask_other_questions
* thanks
  - utter_thanks 
  
## out-of-scope input story              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* out-of-scope
  - utter_out_of_scope
  - utter_ask_other_questions



"""

%store stories_md > stories.md

In [ ]:
train_dialogue()

In [ ]:
load_assistant()

## 2.9 Dialogue model evaluation

Another great way to see how good our dialogue model is, is to test it using evaluation scripts:

In [ ]:

!python -m rasa_core.evaluate --core models/dialogue --stories stories.md -o results


# 3. Closing the feedback loop

Developing an assistant is just one part of the process. Another very important part which defines a successful assistant is enabling your assistant to learn from real user feedback. In the last part of this workshop, we will cover two ways to improve your bots using real user feedback - using interactive learning and using the history of the conversations. We will also, connect our assistant to a custom webpage to see how it works in action! We will complete this part using the command line.

## 3.1. Improving the assistant using the interactive learning 
Interactive learning is a great way to improve your assistant and generate more training example by simply talking to your bot and providing feedback for all predictions it made. That is the main idea behind it - instead of responding right away, an assistant will tell you what it thinks it should do next and ask you for feedback. To start the interactive learning session, we will use a command line and use the following command:


**python -m rasa_core.train interactive --core models/dialogue --nlu models/current/default/nlu --endpoints endpoints.yml**

## 3.2. Storing conversation history 
Another way to improve your assistant is to observe real conversations between the users and a bot. To do so, we have to store the conversations in a storage first. In Rasa Core, tracker is responsible of keeping track of everything that happends throughout the conversation - user inputs, NLU model results, dialogue model predictions, etc. We can easily store all this data in a database for later use. In this step you will learn how to store this information in a Mongo tracker store. We will complete this step in a command line.

First, we will setup a mongodb backend and store the conversaton history there. For that, we will start our assistant on a server using:  
**python -m rasa_core.run -d models/dialogue -u models/current/default/nlu --port 5005  --endpoints endpoints.yml**

##  3.3. Connecting the assistant to the outside world
In the very last step of this workshop, you will learn how to connect your assistant to a custom UI which can be easily added to a website of your choice. The repository of this workshop contains a folder called *bot_ui* where you can find a very basic html webpage. We will add a UI code stored in a *ui.html* file and connect our assistant to it. We will complete this step using a text editor and a command line.

After setting up the backed, we will start our assistant on a server and connect to the UI using:

**python -m rasa_core.run -d models/dialogue -u models/current/default/nlu --port 5005  --endpoints endpoints.yml --credentials credentials.yml**

# 4. Summary 
In this workshop, we covered some of the most important steps of the chatbot development and you should have a simple conference bot running your machine. There are so many things you can do to take this assistant to the whole new level! Here are some ideas for you:
- Add new skills like:
        show the session timetable 
        tell what is the next session at a specific venue   
        connect with a speaker on social media  
        recommend resources to learn more about the topic  
        
- Add new entities like date and time
- Connect your assistant to the most popular messaging platforms like Facebook, Slack or Telegram

Make sure to reference [Rasa official documentation](https://rasa.com/docs) or ask questions on the [Rasa Community Forum](https://forum.rasa.com) if you are in doubt! 

Most importantly, let me know what you came up with!